# 퍼셉트론
 - 뉴런의 출력을 y = f(wx+b) 형태로 나타낸 신경망 모델 (y는 예측값, w는 웨이트(가중치), x는 입력값, b는 바이어스)

# 오차정정학습법
 - 위 식의 정확성을 높이려면 벡터인 웨이트(w)와 바이어스(b)의 최적값을 찾아야 합니다. 오차정정학습법이란 w, b의 수정 폭을 일반식으로 정립하여 w, b의 최적값을 찾아내는 방법입니다.
 - (w의 수정 폭) = (t - y)x
 - (b의 수정 폭) = t - y
 - t는 예측값이 아닌 실제값, 예측값과 실제값의 차를 통해 수정폭을 결정

# 구현 예시
 - 뉴런이 발화하지 않는 데이터는 평균값이 0이고 발화하는 데이터는 평균값이 5이며 각각 10개의 데이터가 있다고 가정하자.

In [24]:
import numpy as np

rng = np.random.RandomState(123) # rng라는 난수 객체 생성, np.random.random은 객체가 아님, 123은 seed

d = 2 # 데이터의 차원
N = 10 # 각 패턴마다의 데이터 수
mean = 5 # 뉴련이 발화하는 데이터의 평균값

x1 = rng.randn(N, d) + np.array([0, 0])
x2 = rng.randn(N, d) + np.array([mean, mean])

x = np.concatenate((x1, x2), axis=0) # 행렬 x1, x2의 가로 합 ex) np.concatenate(([a, b] [b, c]), axis=0) = [a, b, c, d]

In [25]:
w = np.zeros(d)
b = 0
# 웨이트와 바이어스 초기화

In [26]:
def y(x):
    return step(np.dot(w, x) + b)

def step(x):
    return 1 * (x > 0)

# step은 계단함수, 뉴런이 발화하려면 입력값에 대한 wx+b가 임계값을 넘어야 한다. 임계값을 넘었는 지를 판단하기 위해 계단함수 사용

In [27]:
def t(i):
    if i < N:
        return 0
    else:
        return 1
# N개의 뉴련이 발화하면 1, 아니면 0을 출력

In [28]:
while True:
    classified = True
    for i in range(N * 2):
        delta_w = (t(i) - y(x[i])) * x[i]
        delta_b = (t(i) - y(x[i]))
        w += delta_w
        b += delta_b
        classified *= all(delta_w == 0) * (delta_b == 0) # all 은 모든 요소가 참이면 True, 반대인 함수로 any가 있음.
    if classified:
        break
    print(w, b)

[5.73736858 6.49073203] 1
[2.43834346 3.16080635] -4
[6.13951858 4.54533957] -4
[2.44219887 2.99745547] -6
[5.97363736 7.30140141] -6
[4.34630753 3.46317879] -8
[2.14037745 1.2763927 ] -9


In [32]:
print(y([0, 0])) # 발화 x
print(y([5, 5])) # 발화 o

0
1


# 로지스틱 회귀
 - 위에서 보았던 단순 퍼셉트론은 발화한다 안 한다의 이분법적 사고로 0, 1로 구분하였습니다. 하지만 세상 일은 맞다, 아니다로만 판단하기 힘들 때가 있습니다. 예를 들어 회색을 보고 검은색인지 흰색인지 딱 잘라 판단하기 힘든 것처럼 말입니다. 때문에 계단함수(step function)을 대체할 수 있는 활성화 함수(activaition function)이 필요합니다. 이 때 로지스틱 회귀에서 (로지스틱) 시그모이드 함수를 다뤄보겠습니다.
 - sigmoid function = 1 / (1 + exp(-x))
 - 0 < sigmoid function < 1 - 확률론적으로 계산 가능
 - 계단 함수 대신 시그모이드 함수를 사용한 모델을 로지스틱 회귀라 부릅니다.
 - 활성화 함수란(activation function)란 뉴런의 선형결합 후 비선형변환을 진행해주는 함수를 뜻합니다. ex) ReLu, sigmoid, step, etc
 - 시그모이드 함수를 사용하는 이유: (시그모이드 함수 미분값) = (시그모이드 함수)(1 - 시그모이드 함수) 꼴이기 떄문
 - C = 1 일 때, 뉴런이 발화, C = 0 일 때, 뉴런이 발화하지 않는다. - 확률변수라 생각
 - p(C=1 | x) = sigmoid(wx+b)
 - p(C=0 | x) = 1 - p(C=1 | x)
 - y = sigmoid(wx+b)라 생각하면, p(C=t | x) = y^t*(1-y)^(1-t), (t는 0 또는 1)로 식을 나타낼 수 있다.

# 우도 함수
 - x의 데이터가 N개 일 때, 각 x값과 쌍을 이루는 각 t에 대하여 L(w, b) = p(C=t1 | x1) * p(C=t2 | x2) * p(C=t3 | x3) * ... * p(C=tN | xN)
 - L(w, b) = y1^t1*(1-y1)^(1-t1) * y2^t2*(1-y2)^(1-t2) * y3^t3*(1-y3)^(1-t3) * ... * yN^tN*(1-yN)^(1-tN)
 - 최적화(optimizer)를 하려면 우도 함수의 값을 최소나 최대가 되도록 하는 것입니다. 대체로 함수의 최대화는 부호를 반전시키면 최소화가 되므로 일반적으로 함수를 최적화한다라고 할 때는 함수를 최소로 만드는 파라미터를 구하는 것을 의미합니다. - 미분의 개념
 - 다만 우도 함수의 꼴이 다항식의 곱의 꼴이므로 미분하는 것이 까다롭습니다. 따라서 우변항과 좌변항의 로그를 취해 곱셈식을 덧셈식으로 치환합니다. E(w, b) = log(L(w, b)) = -sigma(n=1부터n=N까지)(tnlog(yn) + (1-tn)log(1-yn)) - 교차 엔트로피 함수(cross-entropy error function)
 - 위 함수를 오차 함수 혹은 손실 함수라 부릅니다. ex) MSE(Mean Squared Error)

# 경사하강법
 - 교차 엔트로피 오차 함수에서 입력값 w, b에 대해 w, b를 각각 편미분 해서 0이 되는 값을 구해야 합니다. 하지만 이를 식으로 풀어 구하기는 쉽지 않습니다. 따라서 반복학습을 통해 입력값을 순차적으로 갱신해서 구하는 방법을 사용해야 합니다. (경사하강법)
 - w(k+1번째) = w(k번째) - (학습률)*(E(w,b)를 w로 편미분한 값), b(k+1번째) = b(k번째) - (학습률)*(E(w,b)를 b로 편미분한 값)
 - 학습률이란 하이퍼 파라미터(사용자가 직접 설정해주는 값)로, 모델이 수렴되는 정도를 나타냅니다. 대체로 0.1이나 0.01을 사용합니다. 학습률이 너무 작으면 학습이 오래걸리고 대역최적해가 아닌 국소최적해를 찾을 수도 있다는 단점이 있고, 학습률이 너무 크면 최저점을 무질서하게 이탈할 가능성(수렴하기 힘들어짐)이 있습니다. 
 - 파라미터(w,b)가 더 이상 갱신되지 않는다면 경사가 0이 됐다는 뜻으로 가장 적합한 해를 구했다는 것을 의미합니다.
 - (E(w,b)를 w로 편미분한 값) = -sigma(n=1부터 n=N까지)(tn-yn)xn, (E(w,b)를 b로 편미분한 값) = -sigma(n=1부터 n=N까지)(tn-yn)
 - w(k+1번째) = w(k번째) + (학습률)*(sigma(n=1부터 n=N까지)(tn-yn)xn)), b(k+1번째) = b(k번째) + (학습률)*(sigma(n=1부터 n=N까지)(tn-yn))

# 확률 경사하강법과 미니배치 경사하강법
 - 경사하강법을 통해 로지스틱 회귀를 사용할 수 있지만 파라미터를 갱신할 때마다 N개의 모든 데이터의 합을 구해야 한다는 문제가 있습니다. N이 작을 때는 뭐 큰 문제가 아니지만, 만약 N이 엄청 크다면 메모리가 부족하거나 계산 시간이 엄청 오래 걸립니다. 이 문제를 해결하기 위해 확률 경사하강법을 사용합니다.
 - 확률 경사하강법은 데이터를 하나씩 무작위로 골라서 파라미터를 N번 변경하는 기법입니다.
 - 즉, w(k+1번째) = w(k번째) + (학습률)*(tn-yn)xn), b(k+1번째) = b(k번째) + (학습률)*(tn-yn)로 생각하여 N번 파라미터를 바꿉니다.
 - 이 데이터 전체에 대한 반복 횟수를 epoch(에폭)이라 부릅니다. 1epoch만으로 경사가 0으로 수렴하는 경우는 거의 없으므로 epoch횟수를 적절히 늘려 최적인 해를 찾아야 합니다.
 - 미니배치 경사하강법이란 경사 하강법과 확률 경사하강법의 중간에 존재하는 기법으로 N개의 데이터를 M개씩 쪼개어 학습하는 방법입니다. 보통 M을 50~500정도로 사용하며 메모리의 부족함 없이 선형대수를 연산하여 데이터 연산 속도를 빠르게 할 수 있다는 장점이 있습니다.

# 구현 예시
 - 덴서플로와 케라스를 이용하여 간단한 OR게이트 학습을 예로 들어 구현해보겠습니다.

In [38]:
import numpy as np
import tensorflow as tf

In [39]:
w = tf.Variable(tf.zeros([2,1])) # OR게이트는 입력이 2차원이고 출력이 1차원
b = tf.Variable(tf.zeros([1]))

In [40]:
def y(x):
    return sigmoid(np.dot(w,x) + b)

def sigmoid(x): # Activation function
    return 1/(1 + np.exp(-x))

In [41]:
x = tf.placeholder(tf.float32, shape=[None, 2])
t = tf.placeholder(tf.float32, shape=[None, 1])
y = tf.nn.sigmoid(tf.matmul(x, w) + b) # tf.matmul은 np.dot의 tf버전

In [42]:
cross_entropy = -tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y)) # 손실함수 정의

In [43]:
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)
#오차 함수를 각 파라미터로 편미분해 최적화하는 함수입니다. (0.1은 학습률), 직접 수식으로 계산안 해도 함수를 사용하면 됩니다.

In [46]:
correct_prediction = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)
# y >= 0.5가 발화하는 기준입니다.

In [47]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [1]])
# OR게이트 정의

In [51]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
# 텐서플로는 데이터를 세션이라는 것 안에서 계산합니다. 세션을 만드는 방법은 위와 같습니다.

In [52]:
for epoch in range(200):
    sess.run(train_step, feed_dict={
        x: X,
        t: Y
    })
# 경사하강법을 통해 실제로 학습하는 부분입니다. placeholder에 feed_dict을 사용해 x에 X, t에 Y를 대입합니다.

In [54]:
classified = correct_prediction.eval(session=sess, feed_dict={
    x: X,
    t: Y
})
print(classified)
# 제대로 학습됐는지 확인하는 부분입니다. eval은 텐서에 담겨있는 결과를 볼 수 있는 기본명령어입니다.

[[ True]
 [ True]
 [ True]
 [ True]]


In [55]:
prob = y.eval(session=sess, feed_dict={
    x: X,
    t: Y
})
print(prob)
#각 입력에 대한 발화 확률입니다. OR게이트 의도대로 발화할 것을 알 수 있습니다.(확률이 0.5이상일 때만 발화한다고 위에서 가정함.)

[[0.22355038]
 [0.9142595 ]
 [0.9142595 ]
 [0.9974742 ]]


In [56]:
print('w:', sess.run(w))
print('b:', sess.run(b))
# tf.Variable()로 정의한 변수는 .eval()이 아니라 sess.run을 통해서 구해야 합니다.

w: [[3.6118839]
 [3.6118839]]
b: [-1.2450949]


# 케라스로 구현
 - 텐서플로는 개발자가 직접 수식을 프로그래밍했지만 케라스는 x나 y를 생각할 필요 없이 모델을 정의할 수 있습니다.

In [4]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD

model = Sequential([
    Dense(input_dim=2, units=1), # 입력 2차원, 출력 1차원
    Activation('sigmoid') # 활성화 함수는 sigmoid 함수
])
# model을 미리 Sequential로 만들긴 했지만 .add()함수로 필요한 부분(층)을 추가할 수 있습니다.
# model = Sequential()
# model.add(Dense(input_dim=2, units=1))
# model.add(Activation('sigmoid'))

In [7]:
model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.1)) # SGD(Stochastic Gradient Descent) = 확률 경사하강법

In [8]:
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [1]])

In [9]:
model.fit(X, Y, epochs=200, batch_size=1)


Epoch 1/200
4/4 [==============================] - 0s 96ms/step - loss: 0.5361
Epoch 2/200
4/4 [==============================] - 0s 2ms/step - loss: 0.5049
Epoch 3/200
4/4 [==============================] - 0s 1ms/step - loss: 0.4807
Epoch 4/200
4/4 [==============================] - 0s 2ms/step - loss: 0.4615
Epoch 5/200
4/4 [==============================] - 0s 2ms/step - loss: 0.4453
Epoch 6/200
4/4 [==============================] - 0s 2ms/step - loss: 0.4318
Epoch 7/200
4/4 [==============================] - 0s 5ms/step - loss: 0.4203
Epoch 8/200
4/4 [==============================] - 0s 2ms/step - loss: 0.4103
Epoch 9/200
4/4 [==============================] - 0s 2ms/step - loss: 0.4014
Epoch 10/200
4/4 [==============================] - 0s 4ms/step - loss: 0.3934
Epoch 11/200
4/4 [==============================] - 0s 5ms/step - loss: 0.3864
Epoch 12/200
4/4 [==============================] - 0s 1000us/step - loss: 0.3797
Epoch 13/200
4/4 [==============================] - 0s 2

4/4 [==============================] - 0s 1ms/step - loss: 0.1728
Epoch 102/200
4/4 [==============================] - 0s 2ms/step - loss: 0.1718
Epoch 103/200
4/4 [==============================] - 0s 1000us/step - loss: 0.1707
Epoch 104/200
4/4 [==============================] - 0s 750us/step - loss: 0.1696
Epoch 105/200
4/4 [==============================] - 0s 1000us/step - loss: 0.1686
Epoch 106/200
4/4 [==============================] - 0s 1000us/step - loss: 0.1675
Epoch 107/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1665
Epoch 108/200
4/4 [==============================] - 0s 1ms/step - loss: 0.1655
Epoch 109/200
4/4 [==============================] - 0s 750us/step - loss: 0.1645
Epoch 110/200
4/4 [==============================] - 0s 1000us/step - loss: 0.1635
Epoch 111/200
4/4 [==============================] - 0s 2ms/step - loss: 0.1626
Epoch 112/200
4/4 [==============================] - 0s 1000us/step - loss: 0.1616
Epoch 113/200
4/4 [================

In [11]:
classes = model.predict_classes(X, batch_size=1)
prob = model.predict_proba(X, batch_size=1)
# 학습 후에 나온 결과 담기

In [12]:
print('classified:')
print(Y == classes)
print()
print('output probability:')
print(prob)
# 확률이 위에 꺼랑 조금은 다르지만 거의 유사하게 나온 것을 확인할 수 있습니다.

classified:
[[ True]
 [ True]
 [ True]
 [ True]]

output probability:
[[0.21435973]
 [0.9213187 ]
 [0.91340435]
 [0.9977958 ]]
